In [1]:
import numpy as np
import pandas as pd
import re
from bs4 import BeautifulSoup  #HTML 파싱
from urllib.parse import quote  #검색어 인코딩
import time  #슬립 사용
import requests  #HTML 가져오기
import datetime

In [2]:
jan_vod = pd.read_csv('./LG_data/00. 원본/vod_20240101.csv')
feb_vod = pd.read_csv('./LG_data/00. 원본/vod_20240201.csv')
mar_vod = pd.read_csv('./LG_data/00. 원본/vod_20240301.csv')

jan_vod.info()
feb_vod.info()
mar_vod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7007 entries, 0 to 7006
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   subsr           7007 non-null   int64 
 1   asset_nm        7007 non-null   object
 2   ct_cl           7007 non-null   object
 3   genre_of_ct_cl  7007 non-null   object
 4   use_tms         7007 non-null   int64 
 5   SMRY            7004 non-null   object
 6   ACTR_DISP       7004 non-null   object
 7   disp_rtm        7007 non-null   object
 8   strt_dt         7007 non-null   int64 
dtypes: int64(3), object(6)
memory usage: 492.8+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7770 entries, 0 to 7769
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   subsr           7770 non-null   int64 
 1   asset_nm        7770 non-null   object
 2   ct_cl           7770 non-null   object
 3   genre_of_ct_cl  7770 non-nu

In [5]:
# jan_to_mar_vod = pd.concat([jan_vod, feb_vod, mar_vod])
# jan_to_mar_vod.info() #22557
# concat : index가 중복되는 문제

# 1) df. reset_index(drop=True, inplace = True)
# : reset_index 특성상 기존의 인덱스는 feature로 생성되고
# 새로운 인덱스가 sort되어서 할당 > drop = True 옵션 작성 필요

# 2) df = pd.concat([df1, df2], ignore_index = True)

# Sol---------------------------------------------------------------
#1) 
# jan_to_mar_vod.reset_index(drop = True, inplace = True)
# jan_to_mar_vod

#2) 
jan_to_mar_vod = pd.concat([jan_vod, feb_vod, mar_vod], ignore_index= True)
jan_to_mar_vod


,subsr,asset_nm,ct_cl,genre_of_ct_cl,use_tms,SMRY,ACTR_DISP,disp_rtm,strt_dt
0,66875000,장군재상 10회(A),TV드라마,외화 시리즈,2472,조옥근은 엽소에게 호청의 짝사랑을 전하지만 엽소는 오히려 호청을 찾아가 호되게 나무...,"마사순,성일륜,정천,왕초연,장준명",00:43,20240123085102
1,66705000,수호지의 무송 04회.,TV드라마,외화 시리즈,1891,무대는 무송을 집으로 데려와 회포를 푼다. 반금련은 훤칠하고 잘생긴 무송을 보고 못...,"유대경,번장강,손요기,장한",00:45,20240116203741
2,66705000,수호지의 무송 04회.,TV드라마,외화 시리즈,632,무대는 무송을 집으로 데려와 회포를 푼다. 반금련은 훤칠하고 잘생긴 무송을 보고 못...,"유대경,번장강,손요기,장한",00:45,20240117193153
3,66437000,콩순이 율동교실 4기 01회,키즈,기타,60,콩순이 아이스크림 좋아송. 아이스크림 너무 좋아~! 콩순이와 친구들이 함께 부르는 ...,콩순이,00:01,20240113184221
4,66841000,알사탕,키즈,학습,480,"인기 작가 백희나 신간. 교보문고, YES24 베스트셀러! 상대방의 입장을 헤아리고...",-,00:08,20240113074350
...,...,...,...,...,...,...,...,...,...
22552,66556000,괴물(2023)(예고),영화,드라마,5,싱글맘 사오리는 아들 미나토의 행동에서 이상 기운을 감지한다. 용기를 내 찾아간 학...,"안도 사쿠라,다나카 유코,나가야마 에이타,쿠로카와 소야,히이라기 히나타",00:01,20240314131734
22553,63086000,괴물(2023)(예고),영화,드라마,60,싱글맘 사오리는 아들 미나토의 행동에서 이상 기운을 감지한다. 용기를 내 찾아간 학...,"안도 사쿠라,다나카 유코,나가야마 에이타,쿠로카와 소야,히이라기 히나타",00:01,20240310190508
22554,63017000,만난지3분만에합체 저랑섹스한번하실래요(예고),영화,성인,60,일본 최고의 색녀 시노다 아유미는 마음대로 골라 먹는다. 길거리를 돌아다니며 즉석에...,시노다 아유미,00:01,20240330103945
22555,66056000,만난지3분만에합체 저랑섹스한번하실래요(예고),영화,성인,60,일본 최고의 색녀 시노다 아유미는 마음대로 골라 먹는다. 길거리를 돌아다니며 즉석에...,시노다 아유미,00:01,20240330171645


제목 끝에 . 제거

In [6]:
for idx in jan_to_mar_vod.index :
  jan_to_mar_vod.loc[idx, 'asset_nm'] = re.sub('[.]', '', jan_to_mar_vod.loc[idx, 'asset_nm'])

jan_to_mar_vod.head()

,subsr,asset_nm,ct_cl,genre_of_ct_cl,use_tms,SMRY,ACTR_DISP,disp_rtm,strt_dt
0,66875000,장군재상 10회(A),TV드라마,외화 시리즈,2472,조옥근은 엽소에게 호청의 짝사랑을 전하지만 엽소는 오히려 호청을 찾아가 호되게 나무...,"마사순,성일륜,정천,왕초연,장준명",00:43,20240123085102
1,66705000,수호지의 무송 04회,TV드라마,외화 시리즈,1891,무대는 무송을 집으로 데려와 회포를 푼다. 반금련은 훤칠하고 잘생긴 무송을 보고 못...,"유대경,번장강,손요기,장한",00:45,20240116203741
2,66705000,수호지의 무송 04회,TV드라마,외화 시리즈,632,무대는 무송을 집으로 데려와 회포를 푼다. 반금련은 훤칠하고 잘생긴 무송을 보고 못...,"유대경,번장강,손요기,장한",00:45,20240117193153
3,66437000,콩순이 율동교실 4기 01회,키즈,기타,60,콩순이 아이스크림 좋아송. 아이스크림 너무 좋아~! 콩순이와 친구들이 함께 부르는 ...,콩순이,00:01,20240113184221
4,66841000,알사탕,키즈,학습,480,"인기 작가 백희나 신간. 교보문고, YES24 베스트셀러! 상대방의 입장을 헤아리고...",-,00:08,20240113074350


use_tms < 60초 제거 : vod_over_60

jan_to_mar_vod : under_60 포함

In [12]:
under_60_rows = jan_to_mar_vod[jan_to_mar_vod['use_tms'] < 60] 

vod_over_60 = jan_to_mar_vod.drop(jan_to_mar_vod[(jan_to_mar_vod['use_tms'] < 60)].index, axis= 0) #22557 - 17424

print(vod_over_60[vod_over_60['use_tms'] < 60]) #empty 
print(under_60_rows)

Empty DataFrame
Columns: [subsr, asset_nm, ct_cl, genre_of_ct_cl, use_tms, SMRY, ACTR_DISP, disp_rtm, strt_dt]
Index: []
          subsr                    asset_nm     ct_cl genre_of_ct_cl  use_tms  \
6      65755000            (FREE)섹스 솔루션(무료)        기타             기타       44   
11     60939000      (깜짝무료)설산비호 1999(상) 05회     TV드라마         외화 시리즈       25   
18     60784000             화난 대물에 자지러진 육덕녀        영화             성인       57   
24     62248000                         키싱유        기타             기타       48   
25     66480000                         키싱유        기타             기타       33   
...         ...                         ...       ...            ...      ...   
22546  63419000   덩치 서바이벌-먹찌빠 06회(23/11/12)  TV 연예/오락             기타        0   
22548  64399000  (자막)주술회전 2기: 시부야 사변 16회(N)   TV애니메이션          액션/모험        0   
22550  60784000               길들여진 새댁의 무릎사이        기타             기타       52   
22551  66556000                괴물(2023)(예고)        영화            드라마 

In [10]:
jan_to_mar_vod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22557 entries, 0 to 22556
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   subsr           22557 non-null  int64 
 1   asset_nm        22557 non-null  object
 2   ct_cl           22557 non-null  object
 3   genre_of_ct_cl  22557 non-null  object
 4   use_tms         22557 non-null  int64 
 5   SMRY            22554 non-null  object
 6   ACTR_DISP       22554 non-null  object
 7   disp_rtm        22557 non-null  object
 8   strt_dt         22557 non-null  int64 
dtypes: int64(3), object(6)
memory usage: 1.5+ MB


datetime 형 변환 > 제목 끝 . 제거

In [13]:
jan_to_mar_vod.to_csv(path_or_buf = './LG_data/01. 전처리/vod_preview.csv', sep = ',', header= True,
                    mode= 'a', encoding= 'cp949')

In [20]:
vod_over_60.to_csv(path_or_buf = './LG_data/01. 전처리/vod_전처리_19.csv', sep = ',', header= True,
                    mode= 'a', encoding= 'cp949')

movie / series 분리 > csv 저장

In [ ]:
# over_60
vod_movie = vod_over_60[vod_over_60['ct_cl'] == '영화']
vod_movie.info() #4159

vod_series = vod_over_60[vod_over_60['ct_cl'] != '영화']
vod_series.info() #13265

<class 'pandas.core.frame.DataFrame'>
Index: 4159 entries, 9 to 22555
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   subsr           4159 non-null   int64 
 1   asset_nm        4159 non-null   object
 2   ct_cl           4159 non-null   object
 3   genre_of_ct_cl  4159 non-null   object
 4   use_tms         4159 non-null   int64 
 5   SMRY            4158 non-null   object
 6   ACTR_DISP       4158 non-null   object
 7   disp_rtm        4159 non-null   object
 8   strt_dt         4159 non-null   int64 
dtypes: int64(3), object(6)
memory usage: 324.9+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 13265 entries, 0 to 22556
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   subsr           13265 non-null  int64 
 1   asset_nm        13265 non-null  object
 2   ct_cl           13265 non-null  object
 3   genre_of_ct_cl  13265 non-null  ob

In [15]:
vod_movie.to_csv(path_or_buf = './LG_data/01. 전처리/vod_movie_전처리_19.csv', sep = ',', header= True,
                    mode= 'a', encoding= 'cp949')

vod_series.to_csv(path_or_buf = './LG_data/01. 전처리/vod_series_전처리_19.csv', sep = ',', header= True,
                    mode= 'a', encoding= 'cp949')   

In [16]:
# under_60 포함
vod_movie = jan_to_mar_vod[jan_to_mar_vod['ct_cl'] == '영화']
vod_movie.info() #6163

vod_series = jan_to_mar_vod[jan_to_mar_vod['ct_cl'] != '영화']
vod_series.info() #16394

<class 'pandas.core.frame.DataFrame'>
Index: 6163 entries, 9 to 22555
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   subsr           6163 non-null   int64 
 1   asset_nm        6163 non-null   object
 2   ct_cl           6163 non-null   object
 3   genre_of_ct_cl  6163 non-null   object
 4   use_tms         6163 non-null   int64 
 5   SMRY            6162 non-null   object
 6   ACTR_DISP       6162 non-null   object
 7   disp_rtm        6163 non-null   object
 8   strt_dt         6163 non-null   int64 
dtypes: int64(3), object(6)
memory usage: 481.5+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 16394 entries, 0 to 22556
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   subsr           16394 non-null  int64 
 1   asset_nm        16394 non-null  object
 2   ct_cl           16394 non-null  object
 3   genre_of_ct_cl  16394 non-null  ob

In [17]:
vod_movie.to_csv(path_or_buf = './LG_data/01. 전처리/vod_movie_preview.csv', sep = ',', header= True,
                    mode= 'a', encoding= 'cp949')

vod_series.to_csv(path_or_buf = './LG_data/01. 전처리/vod_series_preview.csv', sep = ',', header= True,
                    mode= 'a', encoding= 'cp949')                    